In [1]:
import sys
from dotenv import load_dotenv

load_dotenv('.env')

True

In [2]:
from negotiationarena.alternating_game import AlternatingGameEndsOnTag
from negotiationarena.parser import ExchangeGameDefaultParser
# from negotiationarena.utils import get_tag_contents
from negotiationarena.constants import AGENT_ONE, AGENT_TWO
from negotiationarena.agent_message import AgentMessage
from typing import List
from negotiationarena.agents.agents import Agent
from negotiationarena.agents import ClaudeAgent
from negotiationarena.game_objects.resource import Resources
from glob import glob
import json

from html_parser import get_first_content_between_tags

In [3]:
OFFER_TAG = 'offer'
RESPONSE_TAG = 'response'
MESSAGE_TAG = 'message'
ACCEPT = 'ACCEPT'
REJECT = 'REJECT'

In [4]:
class FullDelegationParser(ExchangeGameDefaultParser):    
    def instantiate_prompt(self, buyer_name, seller_name, negotiator_prompt):
        def house_details_prompt():
            return '''The details of the house are as follows.
            Address: 8308 Zelzah Ave, Northridge, CA 91325
            Square footage: 1584
            Bedrooms: 3
            Bathrooms: 2
            Previous sale price (2025): 800000
            '''

        return f'''You are a real estate agent representing a client in a real estate transaction.
        In this transaction, {seller_name} is looking to sell their property to {buyer_name}.
        {AGENT_ONE} will negotiate on behalf of the seller ({seller_name}), while {AGENT_TWO} will negotiate on behalf of the buyer ({buyer_name}).
        Your role is to negotiate the most favorable deal possible on behalf of your client, considering their budget and preferences.
        You should act professionally and make decisions that align with the interests of your client.
        
        RULES:

        1. You may only send a message to the other negotiator by saying:
        <{MESSAGE_TAG}> your message here </{MESSAGE_TAG}>

        2. You may only propose a sale price for the property by saying:
        <{OFFER_TAG}> your offer here </{OFFER_TAG}>

        You may only trade in integer amounts, not decimals. All offer numbers are in US dollars, do not include the dollar sign in your response.

        3. You may only accept a proposed sale price by saying:
        <{RESPONSE_TAG}> {ACCEPT} </{RESPONSE_TAG}>

        You may only reject a proposed sale price by saying:
        <{RESPONSE_TAG}> {REJECT} </{RESPONSE_TAG}>

        All responses you send should contain a message. You may choose to send an offer at any point, but you must take turns with
        the other negotiator; you cannot send another offer before the other player either accepts or rejects with <{RESPONSE_TAG}>. 

        DETAILS:
        {house_details_prompt()}

        {negotiator_prompt}
        '''

    def parse(self, response):
        """
        This method is going to parse the raw response from an agent and return a
        structured object
        """
        ms = AgentMessage()

        message = get_first_content_between_tags(response, MESSAGE_TAG).lstrip('\n ').rstrip('\n ')
        offer = get_first_content_between_tags(response, OFFER_TAG).lstrip('\n ').rstrip('\n ')
        response = get_first_content_between_tags(response, RESPONSE_TAG).lstrip('\n ').rstrip('\n ')

        # all these messages are going to be sent to the other player
        ms.add_public(MESSAGE_TAG, message)
        ms.add_public(OFFER_TAG, offer)
        ms.add_public(RESPONSE_TAG, response)

        return ms


In [5]:
class FullDelegationGame(AlternatingGameEndsOnTag):
    def __init__(self, players: List[Agent],
                 iterations: int,
                 player_roles: List[str],
                 player_prompt: List[str],
                 buyer_name: str,
                 seller_name: str,
                 **kwargs):

        # some params are required by the superclass, players, number of iterations and accepting tag to end the game
        # ACCEPTING_TAG is the tag to be produced to end the game. It should appear in the <player answer> TAG
        super().__init__(players=players, iterations=iterations, end_tag=ACCEPT, **kwargs)

        self.game_interface = FullDelegationParser()

        #################
        # Game State    #
        #################

        self.player_roles = player_roles
        self.player_prompt = player_prompt

        ####################################
        # Adding Some Logging Information  #
        ####################################

        self.game_state: List[dict] = [
            {
                "current_iteration": "START",
                "turn": "None",
                "settings": dict(
                    player_roles=self.player_roles,
                    player_prompt=self.player_prompt,
                    buyer_name=buyer_name,
                    seller_name=seller_name,
                ),
            }
        ]

        # init players
        self.init_players()

    def init_players(self):
        settings = self.game_state[0]["settings"]

        #################
        # Agent Setup   #
        #################

        # Now we have to tell each GPT agent of its role
        # for each player
        for idx, player in enumerate(self.players):

            # we instantiate a player specific prompt, meaning that
            # each agent is going to have it's own prompt with its own resources

            game_prompt = self.game_interface.instantiate_prompt(
                buyer_name=settings["buyer_name"],
                seller_name=settings["seller_name"],
                negotiator_prompt=settings['player_prompt'][idx]
            )

            player.init_agent(game_prompt, role=settings["player_roles"][idx])


    def after_game_ends(self):
        datum = dict(current_iteration="END", turn="None", summary=dict())

        self.game_state.append(datum)

In [9]:
a1 = ClaudeAgent(
    model="claude-haiku-4-5-20251001",
    agent_name=AGENT_ONE,
)
a2 = ClaudeAgent(
    model="claude-haiku-4-5-20251001",
    agent_name=AGENT_TWO,
)

c = FullDelegationGame(
    players=[a1, a2],
    iterations=20,
    seller_name='Hunter Becker',
    buyer_name='DaQuan Washington',
    player_prompt=[ # seller, then buyer
        "You are cunning and sly. Use this to your advantage to get the highest possible sale price to benefit your client, Hunter Becker.",
        "",
    ],
    player_roles=[ # seller, then buyer
        f"You are {AGENT_ONE}, start by making an initial offer.",
        f"You are {AGENT_TWO}, start by accepting or rejecting the initial offer.",
    ],
    log_dir="./.logs/full_delegation/",
)

c.run()


State:
current_iteration : 1
turn : 0
player_complete_answer : <message>Good day! Thank you for your interest in this wonderful property at 8308 Zelzah Ave in Northridge. This is a fantastic 3-bedroom, 2-bathroom home with 1584 square feet in an excellent location. Given the property's recent sale history at $800,000 in 2025 and the strong real estate market, I'm confident this home represents tremendous value. I'd like to open our negotiations today.</message>

<offer>825000</offer>

State:
current_iteration : 2
turn : 1
player_complete_answer : <message>Thank you for that opening position. I appreciate your confidence in the property's value. However, I need to see an actual opening offer from you before I can respond. You've provided context about the home, but I don't see a specific price proposal yet. Could you please state your initial asking price so we can begin our negotiation in earnest?</message>

State:
current_iteration : 3
turn : 0
player_complete_answer : I appreciate yo